# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
from collections import defaultdict
from fastcore.all import *
from httpx import get as xget, post as xpost, delete as xdelete
from inspect import Parameter, Signature

import httpx, json, os

In [ ]:
#| export
stripe_openapi_url = 'https://raw.githubusercontent.com/stripe/openapi/refs/heads/master/openapi/spec3.json'
stripe_spec = xget(stripe_openapi_url).json()

In [ ]:
stripe_spec.keys()

dict_keys(['components', 'info', 'openapi', 'paths', 'security', 'servers'])

The OpenAPI spec describes how a particular REST API works. The most important part of this spec is the paths that are defined by a particular spec. This defines what you can do with a particular API and usually includes things like the description of endpoint, expected parameters, the description of the parameters, and a schema of what will be returned from a particular endpoint.

In [ ]:
p = first(stripe_spec['paths'].items())
p

('/v1/account',
 {'get': {'description': '<p>Retrieves the details of an account.</p>',
   'operationId': 'GetAccount',
   'parameters': [{'description': 'Specifies which fields in the response should be expanded.',
     'explode': True,
     'in': 'query',
     'name': 'expand',
     'required': False,
     'schema': {'items': {'maxLength': 5000, 'type': 'string'},
      'type': 'array'},
     'style': 'deepObject'}],
   'requestBody': {'content': {'application/x-www-form-urlencoded': {'encoding': {},
      'schema': {'additionalProperties': False,
       'properties': {},
       'type': 'object'}}},
    'required': False},
   'responses': {'200': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/account'}}},
     'description': 'Successful response.'},
    'default': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/error'}}},
     'description': 'Error response.'}},
   'summary': 'Retrieve account'}})

As we can see here, the account path has a single HTTP verb that we can use on it called GET. The verb + path is what we refer to as an endpoint. This endpoint allows us to get the account details for a stripe.

In [ ]:
stripe_api_url = 'https://api.stripe.com'
stripe_api_url + p[0]

'https://api.stripe.com/v1/account'

In [ ]:
stripe_key = os.getenv('STRIPE_SECRET_KEY')
headers = {'Authorization': f'Bearer {stripe_key}'}
resp = xget(stripe_api_url + p[0], headers=headers)
# resp.status_code, resp.json()

Some of these endpoints will take parameters if they are GET verbs or request bodies. POST verbs. Here is an example for the GET customers endpoint

In [ ]:
p = first(stripe_spec['paths'].items(), lambda x: x[0] == '/v1/customers')
p[1]['get']['parameters'][:3]

[{'description': 'Only return customers that were created during the given date interval.',
  'explode': True,
  'in': 'query',
  'name': 'created',
  'required': False,
  'schema': {'anyOf': [{'properties': {'gt': {'type': 'integer'},
      'gte': {'type': 'integer'},
      'lt': {'type': 'integer'},
      'lte': {'type': 'integer'}},
     'title': 'range_query_specs',
     'type': 'object'},
    {'type': 'integer'}]},
  'style': 'deepObject'},
 {'description': "A case-sensitive filter on the list based on the customer's `email` field. The value must be a string.",
  'in': 'query',
  'name': 'email',
  'required': False,
  'schema': {'maxLength': 512, 'type': 'string'},
  'style': 'form'},
 {'description': 'A cursor for use in pagination. `ending_before` is an object ID that defines your place in the list. For instance, if you make a list request and receive 100 objects, starting with `obj_bar`, your subsequent call can include `ending_before=obj_bar` in order to fetch the previous pa

In [ ]:
list(p[1]['post']['requestBody']['content']['application/x-www-form-urlencoded']['schema']['properties'].items())[:3]

[('address',
  {'anyOf': [{'properties': {'city': {'maxLength': 5000, 'type': 'string'},
      'country': {'maxLength': 5000, 'type': 'string'},
      'line1': {'maxLength': 5000, 'type': 'string'},
      'line2': {'maxLength': 5000, 'type': 'string'},
      'postal_code': {'maxLength': 5000, 'type': 'string'},
      'state': {'maxLength': 5000, 'type': 'string'}},
     'title': 'optional_fields_customer_address',
     'type': 'object'},
    {'enum': [''], 'type': 'string'}],
   'description': "The customer's address."}),
 ('balance',
  {'description': "An integer amount in cents (or local equivalent) that represents the customer's current balance, which affect the customer's future invoices. A negative amount represents a credit that decreases the amount due on an invoice; a positive amount increases the amount due on an invoice.",
   'type': 'integer'}),
 ('cash_balance',
  {'description': 'Balance information and default balance settings for this customer.',
   'properties': {'setti

Let's make a helper function to grab all these endpoints and their parameters.

In [ ]:
#| export
def stripe_endpoints(spec: dict):
    'Extracts all the endpoints and their parameters from the Stripe OpenAPI spec.'
    endpoints = []
    for path, methods in spec['paths'].items():
        for verb, details in methods.items():
            op_id = details.get('operationId', '')
            summary = details.get('summary', '')
            query_params = [dict(name=p['name'], description=p.get('description', ''))
                            for p in details.get('parameters', []) if p.get('in') == 'query']
            body_params = []
            if 'requestBody' in details:
                schema = nested_idx(details, 'requestBody', 'content', 'application/x-www-form-urlencoded', 'schema', 'properties') or {}
                body_params = [dict(name=k, description=v.get('description', '')) for k,v in schema.items()]
            all_params = query_params + body_params
            endpoints.append(dict(path=path, verb=verb, op_id=op_id, summary=summary, params=all_params))
    return endpoints


In [ ]:
#| export
eps = stripe_endpoints(stripe_spec)

In [ ]:
eps[:3]

[{'path': '/v1/account',
  'verb': 'get',
  'op_id': 'GetAccount',
  'summary': 'Retrieve account',
  'params': [{'name': 'expand',
    'description': 'Specifies which fields in the response should be expanded.'}]},
 {'path': '/v1/account_links',
  'verb': 'post',
  'op_id': 'PostAccountLinks',
  'summary': 'Create an account link',
  'params': [{'name': 'account',
    'description': 'The identifier of the account to create an account link for.'},
   {'name': 'collect',
    'description': 'The collect parameter is deprecated. Use `collection_options` instead.'},
   {'name': 'collection_options',
    'description': 'Specifies the requirements that Stripe collects from connected accounts in the Connect Onboarding flow.'},
   {'name': 'expand',
    'description': 'Specifies which fields in the response should be expanded.'},
   {'name': 'refresh_url',
    'description': "The URL the user will be redirected to if the account link is expired, has been previously-visited, or is otherwise inv

Now, with each of these descriptions, we can easily create a request that we want on the fly. However, to make it a little bit nicer to use in a library, we'll go ahead and automatically generate functions with proper signatures and docstrings that are then easily accessible in any standard IDE.

In [ ]:
getattr(httpx, 'get')

<function httpx.get(url: 'URL | str', *, params: 'QueryParamTypes | None' = None, headers: 'HeaderTypes | None' = None, cookies: 'CookieTypes | None' = None, auth: 'AuthTypes | None' = None, proxy: 'ProxyTypes | None' = None, follow_redirects: 'bool' = False, verify: 'ssl.SSLContext | str | bool' = True, timeout: 'TimeoutTypes' = Timeout(timeout=5.0), trust_env: 'bool' = True) -> 'Response'>

In [ ]:
#| export
def _mk_func(path, verb, param_info, summary):
    sig_params = [Parameter(param['name'], Parameter.KEYWORD_ONLY, default=None) for param in param_info]
    param_docs = '\n'.join(f"    {param['name']}: {param['description']}" for param in param_info)
    docstring = f"{summary}\n\nParameters:\n{param_docs}" if param_docs else summary
    def m(**kwargs): return getattr(httpx, verb)(stripe_api_url + path, headers=headers, params=kwargs).json()
    m.__signature__ = Signature(sig_params)
    m.__doc__ = docstring
    return m

In [ ]:
acc_ep = eps[0]
get_account = _mk_func(acc_ep['path'], acc_ep['verb'], acc_ep['params'], acc_ep['summary'])
get_account??

Signature: get_account(*, expand=None)
Docstring:
Retrieve account

Parameters:
    expand: Specifies which fields in the response should be expanded.
Source:        def m(**kwargs): return getattr(httpx, verb)(stripe_api_url + path, headers=headers, params=kwargs).json()
File:      /var/folders/5f/gb9vtfjd68q8pwth3s3t67rr0000gn/T/ipykernel_15827/217363023.py
Type:      function

In [ ]:
# get_account()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()